# Adding Institutions

Here we show how to add institutions to the researchers

In [1]:
import sys
sys.path.append("../")
from MapSci.papers import papers
from MapSci.research_space import research_space

In [2]:
pp = papers("lattes")
maps = research_space("lattes", pp)
maps.compute(2000, 2014)

In [3]:
rs.set_institution("../../dataset/lattes/pesquisadores.csv", sep=";sep;")

We can see that both Professor Alberto Laender and Professor Ana Paula work at UFMG

In [5]:
rs.inst[9089204821424223]

'universidade federal de minas gerais'

In [6]:
rs.inst[2408991231058279]

'universidade federal de minas gerais'

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib.cm as cmx

import sys
sys.path.append("../")
from lib.rs import research_space

In [ ]:
key = "sjr"
rs = research_space(key)
rs.load(2011)
rs.set_institution("../../dataset/lattes/pesquisadores.csv", sep=";sep;")
areas = pd.read_pickle("../../dataset/SJR/areas.pkl")

In [ ]:
rs.advantages()

In [ ]:
insts = [
    "universidade federal de minas gerais",
    "universidade federal de lavras",
    "universidade de sao paulo",
    "universidade federal da paraiba",
    "universidade federal rural de pernambuco",
    "pontificia universidade catolica de minas gerais",
    "petrobras",
    "ministerio da fazenda"
]

In [ ]:
dict_area = areas[["Field", "Classification"]].set_index("Field").to_dict()["Classification"]
unique = areas["Classification"].unique()[1:]
size = Counter(areas["Classification"].tolist())
macro = dict()

for sf, rca in rs.rca_institution.items():
    if rca > 1:
        if (sf[0], dict_area[sf[1]]) in macro:
            macro[(sf[0], dict_area[sf[1]])] += 1 / size[dict_area[sf[1]]]
        else:
            macro[(sf[0], dict_area[sf[1]])] = 1 / size[dict_area[sf[1]]] 

In [ ]:
plt.rcParams["figure.figsize"] = (15,15)
cm = cmx.get_cmap('gnuplot', 26)
color = cm(np.linspace(0, 1, 26))
fig = plt.figure()

for i in range(len(insts)):
    uni = insts[i]
    areas_list = list()
    for a in unique:
        if (uni, a) in macro:
            areas_list.append(macro[(uni, a)])
        else:
            areas_list.append(0)

    sub = fig.add_subplot(4,2,i+1)
    plt.bar(unique, areas_list, color=color, width=1.0)
    plt.xticks(rotation=90)
    plt.title(uni)
    plt.ylabel("Fração de áreas especializadas")
    plt.xlabel("Classificação das áreas")
    if i < 6:
        sub.set_xticks([])
    
fig.tight_layout()
plt.show()

In [ ]:
uni = "universidade federal de lavras"
vals = list()
for f in rs.fields:
    if (uni, f) in rs.rca_institution:
        vals.append(rs.rca_institution[(uni, f)])
    else:
        vals.append(0)

pos = rs.plot(vals)

In [ ]:
def rca_disc(val):
    if val == 0:
        return "Inactive"
    if val < 0.5:
        return "Nascent"
    if val < 1:
        return "Intermediate"
    else:
        return "Developed"

dict_area = {rs.fields[x]: rca_disc(vals[x]) for x in range(len(rs.fields))}
unique = ["Inactive", "Nascent", "Intermediate", "Developed"]
subs = {u: v for v,u in enumerate(sorted(unique))}
values = [subs[dict_area[node]] for node in rs.fields]

pos = rs.plot(values, labels=subs)